In [1]:
import pandas as pd

In [12]:
# load data
df = pd.read_csv('data/cerpen.csv')
df = df.sample(frac=1).reset_index(drop=True)

In [17]:
train_df = df.iloc[:12861, :]
cv_df = df.iloc[12861:17148, :]
test_df = df.iloc[17148:, :]

In [19]:
train_df.to_csv('data/cerpen-training.csv')
cv_df.to_csv('data/cerpen-cross_validation.csv')
test_df.to_csv('data/cerpen-test.csv')

,title,source,authors,categories,text
0,Adios,http://cerpenmu.com/cerpen-perpisahan/adios.html,['Salman Reza Al-Fachrezy'],"['Cerpen Pengalaman Pribadi', 'Cerpen Perpisah...","Lady, begitulah nama dia. \r\nDia adalah anak ..."
1,My Love Story,http://cerpenmu.com/cerpen-cinta-segitiga/my-l...,['Sherly Milenia Islamiati'],['Cerpen Cinta Segitiga'],“anggap saja malam ini kita pacaran” kata kata...
2,Menunggu,http://cerpenmu.com/cerpen-cinta-dalam-hati-te...,['Yulia Nurhasanah'],['Cerpen Cinta Dalam Hati (Terpendam)'],"Mengaguminya dalam diam, setiap detik hanya bi..."
3,Zea dan Persahabatan,http://cerpenmu.com/cerpen-remaja/zea-dan-pers...,['Na'],['Cerpen Remaja'],Pagi ini Zidan bermaksud menyapa Zea. Sekaligu...
4,Hadiah Untuk Rysta,http://cerpenmu.com/cerpen-persahabatan/hadiah...,['Deshinta Maharani'],"['Cerpen Anak', 'Cerpen Persahabatan']","Hai, namaku Shofieya Rika Alyanabila, temanku ..."
